## Publishing package with `uv`

- https://packaging.python.org/en/latest/tutorials/packaging-projects/

- `uv` can be used as part of your publishing process, giving you the ability to generate the `.whl` and `tar.gz` files easily

- We'll use `../write-cli-tools/argparse-greet` as an example package

### Step 1: Set up `pyproject.toml`

- To publish, first thing we want to do is to standardise your `pyproject.toml` 
    - This will be used to determine the package metadata (version, author, etc) 

- Most importantly, we can use pyproject.toml to output the following

```
[project.scripts]
greet = "argparse_greet.main:main"

[build-system]
requires = ["hatchling >= 1.26"]
build-backend = "hatchling.build"

[tool.hatch.build.targets.wheel]
packages = ["src/argparse_greet"]

[[tool.uv.index]]
name = "testpypi"
url = "https://test.pypi.org/simple/"
publish-url = "https://test.pypi.org/legacy/"
explicit = true
```

- `project.scripts` allows you to set cli for your python package. For instance, here I expose a cli command `greet` that looks up the `main()` function in `src/argparse_greet/main.py` 

- `build-system` lets you define which backend build you want to use. Either `hatchling` (for python only projects), or `setuptools` for more complex projects

- `tool.hatch.build.targets.wheel` specifies package path you want to build

- `tool.uv.index` specifies the index to publish to. Note that `[[]]` double braces means you can specify more than 1 index

### Step 2: Build

- It is super easy to build; just use `uv build` with the items specified above

- Build will create a `dist` directory containing the `.whl` and `.tar.gz` of the package

- Note that `build` is **version specific**, so remember to adjust the version parameter in the `pyproject.toml`

- To test locally without publishing, you can run `uv pip install -e .` to install the package in your current virtualenv

### Step 3: Publish

- As of May 2025, `uv publish` does not default to using `~/.pypirc`, where you have your username and password for the publish index specified. You COULD consider swapping to a CI based build, but that will take some effort. For ease, we use `twine` to manage our publishing

- Set up your accounts with the relevant package registries. In the example, we will use `testpypi`

- Set up your `~/.pypirc` file

- Use twine to publish:
```
twine upload --repository testpypi dist/argparse_greet-0.1.4* --verbose
``` 

### Step 4: Test

- To test your package, you can start a fresh environment and run install your package

```
uv pip install --index-url https://test.pypi.org/simple --extra-index-url https://pypi.org/simple argparse-greet
```

- In this case, since your test package is in `testpypi`, remember to supply the actual `pypi` as an extra index, for any dependencies that are missing from `testpypi`